In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
import pingouin as pg
sns.set_theme(style="whitegrid", context="talk")


# Concept module Analysis (VOTC)

## Sub-level preprocess

In [3]:
# 读取数据
df_results = pd.read_csv(r'/data0/user/lxguo/Code/proj_code/scene2/RSA_ROI_Concept_results.csv')
results_list = []

# 被试水平上做平均
for layer in df_results['Layer'].unique():
    for model in df_results['ModelNumber'].unique():
        for roi in df_results['ROIName'].unique():
            
            curr_data = df_results[
                (df_results['Layer'] == layer) & 
                (df_results['ModelNumber'] == model) & 
                (df_results['ROIName'] == roi)
            ]['PartialCorrelation']
            
            if len(curr_data) > 0:  
                mean_rho = np.mean(curr_data)
                # 将结果添加到列表中
                results_list.append({
                    'Layer': layer,
                    'ModelNumber': model,
                    'ROIName': roi,
                    'mean_partial_rho': mean_rho,
                    'n_subjects': len(curr_data)
                })

# 将结果转换为DataFrame
df_meanResults = pd.DataFrame(results_list)
# 将df_stats中的Layer列中包含'CDP'的值替换为'CA'
df_meanResults['Layer'] = df_meanResults['Layer'].str.replace('cdp', 'CA')
df_meanResults


,Layer,ModelNumber,ROIName,mean_partial_rho,n_subjects
0,Concept,1,Reslice3mm_WTPic_Harvord_visual_binary,0.009225,26
1,Concept,2,Reslice3mm_WTPic_Harvord_visual_binary,0.045625,26
2,Concept,3,Reslice3mm_WTPic_Harvord_visual_binary,0.046922,26
3,Concept,4,Reslice3mm_WTPic_Harvord_visual_binary,0.068182,26
4,Concept,5,Reslice3mm_WTPic_Harvord_visual_binary,0.032699,26
5,Concept,6,Reslice3mm_WTPic_Harvord_visual_binary,0.025258,26
6,Concept,7,Reslice3mm_WTPic_Harvord_visual_binary,0.008488,26
7,Concept,8,Reslice3mm_WTPic_Harvord_visual_binary,0.082665,26
8,Concept,9,Reslice3mm_WTPic_Harvord_visual_binary,0.058570,26
9,Concept,10,Reslice3mm_WTPic_Harvord_visual_binary,0.019064,26


## Rename

In [4]:
roi_name_mapping = {
    'Reslice3mm_WTPic_Harvord_visual_binary': 'VOTC',
}
df_meanResults['ROIName'] = df_meanResults['ROIName'].map(roi_name_mapping)

## One-sample ttest

In [5]:
layers = df_meanResults['Layer'].unique()
for idx, layer in enumerate(layers):  
    VOTC_data = df_meanResults[
        (df_meanResults['Layer'] == layer) & 
        (df_meanResults['ROIName'] == 'VOTC')
    ]['mean_partial_rho']
    
    # 计算平均数和标准差
    VOTC_mean = np.mean(VOTC_data)
    VOTC_sd = np.std(VOTC_data)
    
    # 进行单样本T检验
    VOTC_ttest = stats.ttest_1samp(VOTC_data, 0, alternative='greater')

    # 打印T检验结果
    print(f"VOTC - Layer {layer} - 均值: {VOTC_mean:.3f}, 标准差: {VOTC_sd:.3f}, t值: {VOTC_ttest.statistic:.2f}, p值: {VOTC_ttest.pvalue:.4f}")

VOTC - Layer Concept - 均值: 0.035, 标准差: 0.021, t值: 9.27, p值: 0.0000


------------------------------------------

# CA module Analysis (Semantic Control & MD network)

## Sub-level preprocess

In [6]:
# 读取数据
df_results = pd.read_csv(r'/data0/user/lxguo/Code/proj_code/scene2/RSA_ROI_CA_results.csv')
results_list = []

# 被试水平上做平均
for layer in df_results['Layer'].unique():
    for model in df_results['ModelNumber'].unique():
        for roi in df_results['ROIName'].unique():
            
            curr_data = df_results[
                (df_results['Layer'] == layer) & 
                (df_results['ModelNumber'] == model) & 
                (df_results['ROIName'] == roi)
            ]['Correlation']
            
            if len(curr_data) > 0:  
                mean_rho = np.mean(curr_data)
                # 将结果添加到列表中
                results_list.append({
                    'Layer': layer,
                    'ModelNumber': model,
                    'ROIName': roi,
                    'mean_rho': mean_rho,
                    'n_subjects': len(curr_data)
                })

# 将结果转换为DataFrame
df_meanResults = pd.DataFrame(results_list)
df_meanResults


,Layer,ModelNumber,ROIName,mean_rho,n_subjects
0,CA1,1,Reslice3mm_Binary_Jackson2021_semantic_control...,0.015272,26
1,CA1,1,Reslice3mm_Binary_Federenko2013_MultiDemand_ROI,-0.017501,26
2,CA1,2,Reslice3mm_Binary_Jackson2021_semantic_control...,0.045203,26
3,CA1,2,Reslice3mm_Binary_Federenko2013_MultiDemand_ROI,0.036368,26
4,CA1,3,Reslice3mm_Binary_Jackson2021_semantic_control...,0.016415,26
...,...,...,...,...,...
175,CA3,28,Reslice3mm_Binary_Federenko2013_MultiDemand_ROI,0.003410,26
176,CA3,29,Reslice3mm_Binary_Jackson2021_semantic_control...,0.003310,26
177,CA3,29,Reslice3mm_Binary_Federenko2013_MultiDemand_ROI,0.027274,26
178,CA3,30,Reslice3mm_Binary_Jackson2021_semantic_control...,0.013478,26


## Rename

In [7]:
roi_name_mapping = {
    'Reslice3mm_Binary_Jackson2021_semantic_control_ALE_result': 'Semantic Control',
    'Reslice3mm_Binary_Federenko2013_MultiDemand_ROI': 'General Control'
}
# 替换df_stats中的ROI名称 
df_meanResults['ROIName'] = df_meanResults['ROIName'].map(roi_name_mapping)

## One-sample ttest

In [8]:
layers = df_meanResults['Layer'].unique()
for idx, layer in enumerate(layers):  
    if idx != 0: continue
    semantic_data = df_meanResults[
        (df_meanResults['Layer'] == layer) & 
        (df_meanResults['ROIName'] == 'Semantic Control')
    ]['mean_rho']
    
    general_data = df_meanResults[
        (df_meanResults['Layer'] == layer) & 
        (df_meanResults['ROIName'] == 'General Control')
    ]['mean_rho']
    
    # 计算平均数和标准差
    semantic_mean = np.mean(semantic_data)
    semantic_sd = np.std(semantic_data)
    
    general_mean = np.mean(general_data)
    general_sd = np.std(general_data)
    
    # 进行单样本T检验
    semantic_ttest = stats.ttest_1samp(semantic_data, 0, alternative='greater')
    general_ttest = stats.ttest_1samp(general_data, 0, alternative='greater')

    # 打印T检验结果
    print(f"语义控制网络 - 均值: {semantic_mean:.3f}, 标准差: {semantic_sd:.3f}, t值: {semantic_ttest.statistic:.2f}, p值: {semantic_ttest.pvalue:.4f}")
    print(f"一般控制网络 - 均值: {general_mean:.3f}, 标准差: {general_sd:.3f}, t值: {general_ttest.statistic:.2f}, p值: {general_ttest.pvalue:.4f}")


语义控制网络 - 均值: 0.016, 标准差: 0.014, t值: 6.44, p值: 0.0000
一般控制网络 - 均值: 0.009, 标准差: 0.015, t值: 3.22, p值: 0.0016


## ROI comparison

In [9]:
layers = df_meanResults['Layer'].unique()
for idx, layer in enumerate(layers):    
# 进行配对样本T检验
    semantic_data = df_meanResults[
        (df_meanResults['Layer'] == layer) & 
        (df_meanResults['ROIName'] == 'Semantic Control')
    ]['mean_rho']
    
    general_data = df_meanResults[
        (df_meanResults['Layer'] == layer) & 
        (df_meanResults['ROIName'] == 'General Control')
    ]['mean_rho']

    ttest_data = pd.DataFrame({
        'Semantic': semantic_data.values,
        'General': general_data.values
    })
    
    # 均值差和标准差
    diff_data = ttest_data['Semantic'] - ttest_data['General']
    mean_diff = np.mean(diff_data)
    sd_diff = np.std(diff_data)
    
    # paried T检验
    results = pg.ttest(ttest_data['Semantic'], 
                       ttest_data['General'], 
                       paired=True,
                       alternative='two-sided')
    
    # 打印结果
    print(f"\nLayer {layer}:")
    print(results)
    print(f"均值差: {mean_diff:.4f}")
    print(f"差异标准差: {sd_diff:.4f}")


Layer CA1:
               T  dof alternative     p-val        CI95%  cohen-d   BF10  \
T-test  2.888688   29   two-sided  0.007245  [0.0, 0.01]  0.52437  5.922   

           power  
T-test  0.792719  
均值差: 0.0075
差异标准差: 0.0140

Layer CA2:
               T  dof alternative     p-val        CI95%   cohen-d   BF10  \
T-test  2.305486   29   two-sided  0.028488  [0.0, 0.01]  0.426173  1.883   

           power  
T-test  0.616595  
均值差: 0.0063
差异标准差: 0.0148

Layer CA3:
               T  dof alternative    p-val         CI95%   cohen-d   BF10  \
T-test  1.330044   29   two-sided  0.19387  [-0.0, 0.01]  0.235483  0.432   

           power  
T-test  0.238697  
均值差: 0.0036
差异标准差: 0.0145
